# Задание 5.

Загрузите [данные по изменению температуры поверхности земли](https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data). Для этого может понадобится зарегистрироваться на [Kaggle](https://kaggle.com). Затем нужно будет работать с данными, которые содержатся в файле **GlobalLandTemperaturesByMajorCity.csv**

**NB** Все подсчеты необходимо делать с помощью `PySpark`, без применения `pandas api`. Можно использоать `SQL`.

In [5]:
import zipfile
import pandas as pd
from pyspark.sql import SparkSession
import os

spark = SparkSession.builder.master("local[*]").appName("PySpark").getOrCreate()

zip_file = "archive.zip"
file_inside_zip = "GlobalLandTemperaturesByMajorCity.csv"

extracted_dir = "./extracted_files"
os.makedirs(extracted_dir, exist_ok=True)

with zipfile.ZipFile(zip_file) as z:
    z.extract(file_inside_zip, path=extracted_dir)
    extracted_file_path = os.path.join(extracted_dir, file_inside_zip)

    df = spark.read.csv(extracted_file_path, header=True, inferSchema=True)

df.show(10)

import shutil
shutil.rmtree(extracted_dir)

+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|   City|      Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|1849-01-01|            26.704|                        1.435|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-02-01|            27.434|                        1.362|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-03-01|            28.101|                        1.612|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-04-01|             26.14|           1.3869999999999998|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-05-01|            25.427|                          1.2|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-06-01|            24.844|                        1.402|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-07-01|24.058000000000003|                        1.254|Abidjan|Côte

# Задание 4.1 (1 балл)

В последующих заданиях будут учитываться данные начиная с 01.01.1950. Для этого создайте новый `DataFrame`, в котором удалены все строки до 01.01.1950. Используйте созданный DataFrame в последующих заданиях.  

In [15]:
from pyspark.sql.functions import col, to_date, variance

In [17]:
df1 = df.filter(col('dt') >= '1950-01-01')

df1.show(10)

+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|   City|      Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|1950-01-01|26.773000000000003|                        0.239|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1950-02-01|27.526999999999997|                        0.348|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1950-03-01|            28.344|                        0.431|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1950-04-01|             27.83|                        0.467|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1950-05-01|            26.896|                        0.248|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1950-06-01|            25.454|                        0.209|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1950-07-01|            24.878|                        0.403|Abidjan|Côte

# Задание 4.2 (2 балла)

Найдите город, для которого выборочная дисперсия температур на приведенных данных максимальна. 

In [21]:
#выполним группировку данных по городу и вычислим выборочную дисперсию температур
variance_df = df1.groupBy('City').agg(variance('AverageTemperature').alias('TemperatureVariance'))
#сортируем города по возрастанию значений температур и выводим первый
max_variance_city = variance_df.orderBy(col('TemperatureVariance').desc()).select('City').first()[0]
print(max_variance_city)

Harbin


# Задание 4.3 (2 баллов)

Посчитайте данные по среднегодовой температуре в Санкт-Петербурге. Определите года, в которых средняя температура была выше, чем в предыдущем  и последующем году. 

In [27]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [29]:
#берём данные для Санкт-Петербурга и извлекаем год
spb_df = df1.filter(df1.City == 'Saint Petersburg')
spb_df = spb_df.withColumn('Year', F.year(F.to_date('dt')))

#вычисляем среднегодовую температуру для каждого года в Санкт-Петербурге
avg_temp_yearly = spb_df.groupBy('Year').agg(F.avg('AverageTemperature').alias('AvgTemperature'))
#вычисляем среднюю температуру в предыдущем и следующем году
windowSpec = Window.orderBy('Year')
avg_temp_yearly = avg_temp_yearly.withColumn('PrevYearAvgTemp', F.lag('AvgTemperature').over(windowSpec))
avg_temp_yearly = avg_temp_yearly.withColumn('NextYearAvgTemp', F.lead('AvgTemperature').over(windowSpec))

#выбираем года, в которых средняя температура была выше, чем в предыдущем и последующем году
higher_than_prev_and_next = avg_temp_yearly.filter((avg_temp_yearly['AvgTemperature'] > avg_temp_yearly['PrevYearAvgTemp']) &
                                                   (avg_temp_yearly['AvgTemperature'] > avg_temp_yearly['NextYearAvgTemp']))
higher_than_prev_and_next.select('Year', 'AvgTemperature').orderBy('Year').show()

+----+------------------+
|Year|    AvgTemperature|
+----+------------------+
|1953| 4.840083333333333|
|1957| 4.856249999999999|
|1959| 5.026000000000001|
|1961| 5.842083333333334|
|1964| 4.414166666666666|
|1967| 4.618666666666667|
|1972| 5.294333333333333|
|1975| 5.885249999999999|
|1977|3.9462499999999996|
|1979|4.0605833333333345|
|1983| 5.237583333333333|
|1986| 4.054749999999999|
|1989| 6.585916666666667|
|1992| 5.415833333333334|
|1995| 5.422583333333333|
|1997| 4.668416666666667|
|2000| 6.184666666666665|
|2002|           5.07675|
|2005| 5.553249999999999|
|2008|6.4084166666666675|
+----+------------------+
only showing top 20 rows



# Задание 4.4 (4 балла)

Найдите города, для которых: 
1. Разница между максимальным и минимальным значением среднегодовой температуры в выборке максимальна.
2. Самая большая средняя разница между средней температурой января и средней температурой июля.
3. Наибольшее среднее количество месяцев с отрицательной температурой в году.